<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/Test(CNN_bi_SA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
filtered_df = pd.read_csv('/content/drive/MyDrive/preprocessed_tweets.csv')
filtered_df.head()

,Unnamed: 0,id,dialect,tweets
0,15497,1009754958479151232,1,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...
1,15498,1009794751548313600,1,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...
2,15499,1019989115490787200,1,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...
3,15500,1035479791758135168,1,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...
4,15501,1035481122921164800,1,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...


In [ ]:
filtered_df.dialect.value_counts()

1    36499
0    16183
2    11539
3     9246
Name: dialect, dtype: int64

In [ ]:
filtered_df = filtered_df.dropna()

In [ ]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [ ]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [ ]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)

In [ ]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
# Convert data to PyTorch tensors
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

vocab_size = 20000
embedding_dim = 100
hidden_dim = 128
output_dim = 4
num_layers = 2
bidirectional = True
lr = 0.01
batch_size = 64
num_epochs = 100
patience = 5

class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, embedding_dim), padding=(1, 0))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), padding=(1, 0))
        self.lstm = nn.LSTM(200, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=0.5, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.attention = SelfAttention(hidden_dim * 2)

    def forward(self, x):
        embedded = self.embedding(x)
        conv1_out = F.relu(self.conv1(embedded.unsqueeze(1)))
        conv2_out = F.relu(self.conv2(conv1_out))
        conv2_out = conv2_out.permute(0, 2, 1, 3)
        conv2_out = conv2_out.reshape(conv2_out.size(0), conv2_out.size(1), -1)
        lstm_in = conv2_out.permute(0, 2, 1)
        lstm_out, _ = self.lstm(lstm_in)
        lstm_out = self.dropout(lstm_out)
        attention_out , attention_weights= self.attention(lstm_out)
        fc_out = self.fc(attention_out)
        return fc_out, attention_weights

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, encoder_outputs):
        energy = self.projection(encoder_outputs)
        weights = F.softmax(energy.flatten(start_dim=1), dim=1)
        outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
        return outputs, weights

In [ ]:
import torch
model = BiLSTMAttention(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers)
model.load_state_dict(torch.load("/content/model2.pth", map_location=torch.device('cpu')))
# model.to("cuda:0")
model.eval()
X_test = X_test
# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred_ = model(X_test)
    print(pred_)

(tensor([[-0.9055, -1.0567, -1.1945, -1.1836],
        [-0.9055, -1.0567, -1.1945, -1.1836],
        [-0.9055, -1.0567, -1.1945, -1.1836],
        ...,
        [-0.9055, -1.0567, -1.1945, -1.1836],
        [-0.9055, -1.0567, -1.1945, -1.1836],
        [-0.9055, -1.0567, -1.1945, -1.1836]]), tensor([[1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21],
        [1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21],
        [1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21],
        ...,
        [1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21],
        [1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21],
        [1.9252e-03, 6.8995e-03, 2.5312e-02,  ..., 2.6256e-02, 9.3752e-05,
         2.5925e-21]]))


In [ ]:
X_test.size()

torch.Size([7344, 200])

In [ ]:
print(len(pred_))

2


In [ ]:
preds = np.argmax(pred_[0].cpu().numpy(), axis=1)

In [ ]:
def one_hot(a, num_classes):
   a = np.array(a) 
   return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
preds = one_hot(preds,4)

In [ ]:
preds
pred_results_np = np.array(preds)
x_np = torch.from_numpy(pred_results_np)

In [ ]:
len(x_np)

7344

In [ ]:
y_test.size()

torch.Size([7344, 4])

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
recall_score(y_test, x_np,average='macro') 

0.25

In [ ]:
precision_score(y_test, x_np,average='macro') 

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0542619825708061

In [ ]:
accuracy_score(y_test, pred_results_np)

0.2170479302832244

In [ ]:
f1_score(y_test, x_np,average='macro')

0.08916983665249496

In [ ]:
# Set the model to evaluation mode
model.eval()
device = "cuda"
# Tokenize the new complaint and pad the sequence
new_complaint = ['واش دير للعشى']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=max_length)

# Convert the padded sequence to a PyTorch tensor and move it to the device (e.g., GPU) if available
padded_tensor = torch.LongTensor(padded)

# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred = model(padded_tensor)

# Move the prediction back to the CPU and convert to a numpy array
#pred = pred.cpu().numpy()

# Map the prediction to a class label using the CLASS_DICT{'DZ': 0, 'LY': 1, 'MA': 2, 'TN': 3}
CLASS_DICT = {0: "DZ",1: "LY", 2: "MA",3: "TN"}
class_label = CLASS_DICT[np.argmax(pred[0].cpu().numpy(), axis=1)[0]]

# Print the prediction and the predicted class label
print(class_label)

DZ
